# MLFlow Service

In dieser Übung setzen wir den MLFlow Dienst auf.

# Vorbereitung

Diesmal gibt es nichts vorzubereiten.

# Übungen

Wir erweitern unsere Infrastruktur um MLFlow, welches und Experiment Tracking, ein Deployment API und eine Model Registry zur Verfügung stellt.

Wir hatten in der ersten Übung die beiden Docker Netzwerke `development` und `production` definiert. In welches dieser Netze binden wir MLFlow ein?

Richtig, in beide natürlich. MLFlow muss während der Entwicklung für das Logging von Modellen verfügbar sein, und wärend dem Betrieb für den Bezug von ML Modellen aus der Registry.

----

Schau dir das compose file von MLFlow an.
 * Via welchen Port kannst Du mit auf MLFlow zugreifen?
 * Wo sind die Artefakte gespeichert?
 * Wo sind die Metadaten gespeichert? 

Die Antwort findest du in den Direktiven command, ports und volume im compose file:

    command: mlflow server --host 0.0.0.0 --port 5001 --artifacts-destination s3://artifacts --backend-store-uri sqlite:///metadata.db

    ports:
      - 5001:5001  # 5000 is taken by airplay on macos
      
    volumes:
      - ../../mlflow_metadata.sqlite:/metadata.sqlite

----

Aktiviere und starte nun den MLFlow Dienst.

Du führst dazu die folgenden fünf Schritte aus:

1. Alle laufenden Notebooks speichern
1. Alle Container stoppen (entweder mit ctrl-c wenn der Docker Prozess im Vordergrund läuft oder mit `docker compose down`) im Verzeichnis `/exercises`, wo das Top-Level File `docker-compose.yml` liegt.
1. Im top-level compose file die Zeile mit MLFlow einkommentieren:

````
include:
  - docker_includes/networks.yml
  - containers/minio.yml
  - containers/development_env/development_env.yml
  - containers/mlflow/mlflow.yml
#  - containers/message-broker/message-broker.yml
#  - containers/monitoring/monitoring.yml
````

4. Im compose File der Development Umgebung (`containers/development_env/development_env.yml`) die Umgebungsvariable MLFLOW_TRACKING_URI einkommentieren:
````
      environment:
        - FSSPEC_S3_KEY=${MINIO_ADMIN}
        - FSSPEC_S3_SECRET=${MINIO_SECRET}
        - FSSPEC_S3_ENDPOINT_URL=${MINIO_ENDPOINT}
        - GIT_PYTHON_REFRESH=quiet
        - MLFLOW_TRACKING_URI=http://model-registry:5001
````
5. Die container mit `docker-compose up` oder `docker-compose up -d` wieder starten

----

Unsere Umgebung sieht nun so aus:

![summary.png](summary_mlflow.png)

----

Schau dir nochmals compose file von MLFlow an. Etwas gibt es noch zu tun, kannst Du herausfinden, was?

Wir starten MLFlow mit dem Befehl

`command: mlflow server --host 0.0.0.0 --port 5001 `**--artifacts-destination s3://artifacts**` --backend-store-uri sqlite:///metadata.db`

Dieser Bucket im Object Store muss manuell erstellt werden. Alles, was darunter liegt, erstellt und verwaltet MLFlow dann selber.

Erstelle den Bucket.

In [ ]:
import s3fs

s3fs.S3FileSystem().mkdir('/artifacts')

----

**Bitte quittiere wiederum auf [Mentimeter](https://www.menti.com/alaxbnek73eu), dass du mit der Übung durch bist**.

Du kannst nun gleich zur nächsten Übung (`10_Ein_einfaches_Modell_mit_MLFlow.ipynb`) gehen.